# Indiahacks machine learning challenge
### Predict the segment
https://www.hackerearth.com/problem/machine-learning/predict-the-segment-hotstar/

In [1]:
import numpy as np
import pickle
import pandas as pd
import re
from xgboost import XGBClassifier as XGB
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import GradientBoostingClassifier as GB
from sklearn.ensemble import ExtraTreesClassifier as ETC
from sklearn.preprocessing import PolynomialFeatures as PF
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.preprocessing import MinMaxScaler,PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingClassifier as VC
from sklearn.ensemble import AdaBoostClassifier as ADA
from sklearn.ensemble import IsolationForest as ISO
from sklearn.feature_selection import SelectKBest as SK

In [2]:
file = open("train.pickle","rb")
train_data = pickle.load(file)
file.close()
file = open("test.pickle","rb")
test_data = pickle.load(file)
file.close()
file = open("target.pickle","rb")
target = pickle.load(file)
file.close()
test_d = pd.read_json('test_data.json',orient='index')
test_d.reset_index(inplace = True)
test_d.rename(columns={'index':'ID'}, inplace=True)
test_id = test_d['ID']
test_data = test_data[train_data.columns]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, target,train_size = 0.5,random_state = 0)
#train_test_scaled
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
test_scaled = scaler.transform(test_data)
y_train = np.array(y_train)
from sklearn.feature_selection import SelectFpr as SK
sk = SK(alpha=0.05)
X_train_feat = sk.fit_transform(X_train_scaled,y_train)
X_test_feat = sk.transform(X_test_scaled)
test_feat = sk.transform(test_scaled)
print(X_train_feat.shape)

(100000, 37)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [25] are constant.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:541: RuntimeWarning: invalid value encountered in less
  return self.pvalues_ < self.alpha


In [ ]:
ISO_clf = ISO(n_estimators=500,random_state=0)
ISO_clf.fit(X_train,y_train)
pred = ISO_clf.decision_function(X_train)

In [ ]:
len(pred[pred>0.1])

In [ ]:
to_del = []
for i,p in enumerate(pred>0.1):
    if p==True:
        to_del.append(i)
for i,p in enumerate(pred<-0.02):
    if p==True:
        to_del.append(i)
X_train = np.array(X_train)
X_test = np.array(X_test)
test_data = np.array(test_data)
X_train = np.delete(X_train,to_del,axis=0)
y_train = np.delete(y_train,to_del)
print(len(X_train),len(y_train))

In [28]:
XGB_clf = XGB(n_estimators=100,max_depth=5,random_state=0)
XGB_clf.fit(X_train_feat,y_train)
print(roc_auc_score(y_train,XGB_clf.predict_proba(X_train_feat)[:,1]))
print(roc_auc_score(y_test,XGB_clf.predict_proba(X_test_feat)[:,1]))

0.824225052527
0.801324441485


In [ ]:
pos = X_train[y_train==1]
neg = X_train[y_train==0]
#print(pos['title_sum'],neg['title_sum'])

In [ ]:
from xgboost import plot_importance,plot_tree
import matplotlib.pyplot as plt
plot_importance(XGB_clf)
plt.show()

In [ ]:
from sklearn.svm import SVC as SV
SV_clf = SV(random_state=0)
SV_clf.fit(X_train_feat,y_train)
print(roc_auc_score(y_test,SV_clf.predict_proba(X_test_feat)[:,1]))

In [47]:
from sklearn.linear_model import LogisticRegression as LR
LR_clf = LR(C=100,max_iter=1000,penalty='l2',random_state=0)
LR_clf.fit(X_train_feat,y_train)
print(roc_auc_score(y_test,LR_clf.predict_proba(X_test_feat)[:,1]))

0.790111893761


In [ ]:
RF_clf = RF(n_estimators=100,max_depth=12,random_state=0)
RF_clf.fit(X_train,y_train)
print(roc_auc_score(y_test,RF_clf.predict_proba(X_test)[:,1]))

In [ ]:
MLP_clf = MLP(hidden_layer_sizes=[1,100,10],alpha=0.0001,solver='lbfgs',random_state=0)
MLP_clf.fit(X_train_scaled,y_train)
print(roc_auc_score(y_test,MLP_clf.predict_proba(X_test_scaled)[:,1]))

In [ ]:
ADA_clf = ADA(n_estimators=100,random_state=0)
ADA_clf.fit(X_train,y_train)
print(roc_auc_score(y_test,ADA_clf.predict_proba(X_test)[:,1]))

In [49]:
XGB_clf = XGB(n_estimators=100,max_depth=5,random_state=0)
RF_clf = RF(n_estimators=100,max_depth=5,random_state=0)
GB_clf = GB(n_estimators=100,max_depth=5,random_state=0)
LR_clf = LR(C=100,max_iter=1000,penalty='l2',random_state=0)
#MLP_clf = MLP(hidden_layer_sizes=[100,10,10],solver='adam',random_state=0)
ADA_clf = ADA(n_estimators=100,random_state=0)
ETC_clf = ETC(n_estimators=100,max_depth=5,random_state=0)

In [50]:
#vc_clf = VC(estimators=[('RF',RF_clf),('XGB',XGB_clf),('GB',GB_clf),('ADA',ADA_clf),('ETC',ETC_clf)],
 #          voting='soft', weights=[1,2,1,1,1])
vc_clf = VC(estimators=[('RF',RF_clf),('XGB',XGB_clf),('ADA',ADA_clf),('GB',GB_clf),('ETC',ETC_clf)],
           voting='soft', weights=[1,1,1,1,1])
vc_clf.fit(X_train_feat,y_train)
print(roc_auc_score(y_test,vc_clf.predict_proba(X_test_feat)[:,1]))
print(roc_auc_score(y_train,vc_clf.predict_proba(X_train_feat)[:,1]))

0.799168689746
0.815599918456


In [33]:
pred = vc_clf.predict_proba(test_feat)

In [34]:
#make submission file and submit
columns = ['segment']
sub = pd.DataFrame(data=pred[:,1], columns=columns)
sub['ID'] = test_id
sub = sub[['ID','segment']]
sub.to_csv("sub_hot.csv", index=False,float_format='%0.7f')